# Input file requirements:
`./CT-data.dat`

In [ ]:
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as pp
import pandas as pd
import matplotlib as mpl
from matplotlib import gridspec
from MDAnalysis.analysis.rms import RMSD
from scipy import stats
from IPython.display import Markdown


%matplotlib notebook
print(mpl.__version__)

In [ ]:
def get_density(vals, *args, **kwargs):
    kernel = stats.gaussian_kde(vals, *args, **kwargs)

    x = np.linspace(min(vals), max(vals), 1000)
    y = kernel(x)

    return x, y

In [ ]:
df_ct = pd.read_csv('trajectories/MD-1ZNF/CT-data.dat', sep='\s+', skiprows=[1])
SimTime = 40
dt = SimTime/df_ct.shape[0]

df_ct['Time'] = np.arange(dt, SimTime+0.5*dt, dt)

dist_files = {
    at:f'trajectories/MD-1ZNF/{at}-Zn-dist.dat' for at in 'N1 N2 S1 S2'.split()
}

inds = []
for n, (at, dist_file) in enumerate(dist_files.items()):
    inds.extend([(at, 'CTPOL'), (at, 'opt-CTPOL')])
    _df = pd.read_csv(
        dist_file, sep='\s+', 
        skiprows=1, 
        names=[f'CTPOL_{at}', f'opt-CTPOL_{at}'],
        
    )
    _df.index = np.arange(dt, SimTime+0.5*dt, dt)
    if n == 0:
        df_dist = _df
    else:
        df_dist = pd.merge(df_dist, _df, left_index=True, right_index=True)

        
display(df_ct)
df_dist.columns = pd.MultiIndex.from_tuples(inds, names=['Atom', 'FF'])
df_dist.index.name = 'Time'
display(df_dist)


In [ ]:
df_dist_ctpol = df_dist.xs("CTPOL", level="FF", axis=1)
df_dist_opt = df_dist.xs("opt-CTPOL", level="FF", axis=1)

nums = '0123456789'

mean_df = df_dist_ctpol.groupby(
    by=lambda x: x.strip(nums),
    axis=1
).mean()



In [ ]:
fig, ax = pp.subplots()
fig.set_size_inches(7,4)

ax2 = ax.twinx()
ax.set_zorder(ax2.get_zorder()+1)
ax.patch.set_visible(False)

df_ct.plot(
    x="Time", y="CTPOL", 
    ax=ax2, 
    alpha=0.5, 
    label="Charge Transfer",
    zorder=1,
    legend=False)

mean_df.plot(ax=ax, color='green orange'.split(), legend=False)

ax.set_xlim(0,40)
ax.set_ylabel('Distance (nm)')
ax2.set_ylabel('Charge Transfer')
ax.set_xlabel("Time (ns)")

leg = ax.legend()
for lh in leg.legendHandles: 
    lh.set_alpha(1)

In [ ]:

fig = pp.figure()
fig.set_size_inches(8,5)
gs = gridspec.GridSpec(1, 3)
gs.update(hspace=0.01, wspace=0.01)



ax_left = fig.add_subplot(gs[0, 0:2])
ticks = ax_left.get_xticks

ax_right = fig.add_subplot(gs[0, 2], sharey=ax_left)


ax_left.set_xlabel(r"Time (ns)", fontsize=16)

ax_right.yaxis.set_visible(False)
ax_right.xaxis.set_visible(False)

'''
ax_left.text(
    0.5, 0.7, sel[0], 
    horizontalalignment='left',
    verticalalignment='bottom',
    transform=ax_left.transAxes,
    fontsize=14
)
'''
df_ct.plot(
    x='Time',
    ax=ax_left
)

for pol in df_ct:
    if pol == 'Time': continue
    x, y = get_density(df_ct[pol], bw_method=0.1)
    
    ax_right.plot(y, x, label=pol)


ax_right.set_xlim(left=0)
ax_left.set_xlim(min(df_ct.Time), max(df_ct.Time)) 

ax_right.legend()
fig.text(
    0.04, 0.5, r"Charge transfer [e]", 
    va='center', rotation='vertical', fontsize=16
)

fig.tight_layout()
pp.show()

In [ ]:
df_outlier = mean_df.N[mean_df.N > 0.3]

times = df_outlier.index

exceeds_at = []
cur_int = 0
for t in times:
    if int(t) > cur_int:
        cur_int = int(t)
        exceeds_at.append(t % 1)
        
exceeds_at